In [1]:
import pandas as pd
import nltk
import re

D:\SmartInternz\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\SmartInternz\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\SmartInternz\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paint\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
ps = PorterStemmer()

In [4]:
data = []

In [5]:
df = pd.read_csv("IMDB Dataset.csv", delimiter = ",")

In [6]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [7]:
for i in range(0,5000):
    review = df['review'][i]
    #removal of unneccesary characters
    review = re.sub('[^a-zA-Z]',' ', review)
    #text converted to lower case
    review = review.lower()
    #splitting sentences into words
    review = review.split()
    #stemming and remove stopwords
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    data.append(review)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 10000)
cv

CountVectorizer(max_features=10000)

In [9]:
x = cv.fit_transform(data).toarray()
y = df.iloc[:5000,1:2].values

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([("one",OneHotEncoder(),[0])],remainder = "passthrough")

In [11]:
y = ct.fit_transform(y)

In [13]:
import numpy as np
x = np.asarray(x).astype(np.float32)

In [16]:
import tensorflow
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

In [17]:
model = Sequential()

In [18]:
model.add(Dense(1565, activation = "relu"))
model.add(Dense(3000, activation = "relu"))
model.add(Dense(2, activation = "softmax"))

In [19]:
model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ['accuracy'])

In [20]:
model.fit(x_train,y_train,epochs=20)

Epoch 1/20
125/125 [==============================] - 10s 66ms/step - loss: 0.4309 - accuracy: 0.8167
Epoch 2/20
125/125 [==============================] - 8s 66ms/step - loss: 0.0472 - accuracy: 0.9835
Epoch 3/20
125/125 [==============================] - 8s 66ms/step - loss: 0.0084 - accuracy: 0.9975
Epoch 4/20
125/125 [==============================] - 8s 66ms/step - loss: 0.0117 - accuracy: 0.9958
Epoch 5/20
125/125 [==============================] - 8s 66ms/step - loss: 0.0018 - accuracy: 0.9995
Epoch 6/20
125/125 [==============================] - 8s 66ms/step - loss: 5.4667e-05 - accuracy: 1.0000
Epoch 7/20
125/125 [==============================] - 8s 67ms/step - loss: 1.9813e-05 - accuracy: 1.0000
Epoch 8/20
125/125 [==============================] - 9s 69ms/step - loss: 1.4472e-05 - accuracy: 1.0000
Epoch 9/20
125/125 [==============================] - 8s 67ms/step - loss: 1.0856e-05 - accuracy: 1.0000
Epoch 10/20
125/125 [==============================] - 8s 67ms/step - loss

In [21]:
model.save("Movie_Reviewer.h5")

In [29]:
review = "The movie was amazing"
review = re.sub('[^a-zA-Z]', ' ', review)
review = review.lower()
review = review.split()
review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
review =' '.join(review)

In [30]:
pred = model.predict(cv.transform([review]))
sentiment = ["negative", "positive"]
prediction = sentiment[np.argmax(pred)]
prediction

'positive'